In [8]:
import pandas as pd

In [9]:
!dir

 Volume in drive C is Windows-SSD
 Volume Serial Number is 8A4D-3C00

 Directory of C:\Code\CodingJourney\AIJourney\Project_01_ai_finance_advisor\tests

11-12-2025  16:05    <DIR>          .
07-12-2025  21:45    <DIR>          ..
07-12-2025  20:45           247,296 Acct Statement_8340_07122025_20.42.57.xls
11-12-2025  16:05             1,107 train_classifier.ipynb
               2 File(s)        248,403 bytes
               2 Dir(s)  13,724,119,040 bytes free


In [10]:
df = pd.read_excel("Acct Statement_8340_07122025_20.42.57.xls")

In [11]:
df

,HDFC BANK Ltd. Page No .: 1 Statement of accounts,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Account Branch :SWASTIK VIHAR,NaN,NaN
4,MR. DEVARSH RAJ,NaN,NaN,NaN,Address :HDFC BANK LTD,NaN,NaN
...,...,...,...,...,...,...,...
806,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,State account branch GSTN:,06AAACH2702H1Z4,NaN,NaN,NaN,NaN,NaN
808,HDFC Bank GSTIN number details are available a...,NaN,NaN,NaN,NaN,NaN,NaN
809,"Registered Office Address: HDFC Bank House, Se...",NaN,NaN,NaN,NaN,NaN,NaN
